# Proyecto Rusty Bargain Carseller

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Carga y exploración de los datos

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import time
from catboost import CatBoostRegressor, Pool


In [2]:
df = pd.read_csv('car_data.csv')

In [3]:
print("📦 Columnas y tipos de datos:\n", df.dtypes)
print("\n🔁 Duplicados encontrados:", df.duplicated().sum())
print("\n🧱 Valores nulos por columna:\n", df.isna().sum())
print("\n📊 Estadísticas descriptivas generales:\n", df.describe(include='all'))

📦 Columnas y tipos de datos:
 DateCrawled          object
Price                 int64
VehicleType          object
RegistrationYear      int64
Gearbox              object
Power                 int64
Model                object
Mileage               int64
RegistrationMonth     int64
FuelType             object
Brand                object
NotRepaired          object
DateCreated          object
NumberOfPictures      int64
PostalCode            int64
LastSeen             object
dtype: object

🔁 Duplicados encontrados: 262

🧱 Valores nulos por columna:
 DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Mileage                  0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64



## Preparación de datos

In [4]:
# Esta preparación es unicamente para el modelo RandomForest
# 1. Eliminar columnas irrelevantes
df = df.drop(['DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures', 'PostalCode'], axis=1)

# 2. Rellenar valores nulos
for col in ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']:
    df[col] = df[col].fillna('unknown')

# 3. Convertir NotRepaired a binaria
df['NotRepaired'] = df['NotRepaired'].map({'yes': 1, 'no': 0, 'unknown': -1})

# 4. Separar variables categóricas y numéricas
categorical = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand']
numerical = ['RegistrationYear', 'Power', 'Mileage', 'RegistrationMonth', 'NotRepaired']

# Codificación categórica para árboles simples (se puede cambiar según el modelo)
encoder = OrdinalEncoder()
df[categorical] = encoder.fit_transform(df[categorical])

# 5. Separar features y target
X = df.drop('Price', axis=1)
y = df['Price']

# 6. División del conjunto
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


## Entrenamiento del modelo 

In [5]:
# 🌳 Entrenamiento con Random Forest Regressor

# 1. Inicializar el modelo con hiperparámetros base
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1  # Utilizar todos los núcleos disponibles
)

# 2. Medir tiempo de entrenamiento
start_train = time.time()
rf_model.fit(X_train, y_train)
end_train = time.time()

# 3. Medir tiempo de predicción
start_pred = time.time()
y_pred = rf_model.predict(X_valid)
end_pred = time.time()

# 4. Evaluar con RECM (Raíz del Error Cuadrático Medio)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))

# 5. Mostrar resultados
print("🌳 Random Forest Regressor")
print(f"🔧 Hiperparámetros: n_estimators=100, max_depth=10")
print(f"🧠 RECM (validación): {rmse:.2f}")
print(f"⏱️ Tiempo de entrenamiento: {end_train - start_train:.2f} s")
print(f"⏱️ Tiempo de predicción: {end_pred - start_pred:.4f} s")



🌳 Random Forest Regressor
🔧 Hiperparámetros: n_estimators=100, max_depth=10
🧠 RECM (validación): 2044.23
⏱️ Tiempo de entrenamiento: 25.67 s
⏱️ Tiempo de predicción: 0.2765 s


In [7]:
# 1. Recarga de datos original sin codificación previa
df_cb = pd.read_csv('car_data.csv')

# 2. Eliminar columnas irrelevantes
df_cb = df_cb.drop(['DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures', 'PostalCode'], axis=1)

# 3. Rellenar valores nulos
for col in ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']:
    df_cb[col] = df_cb[col].fillna('unknown')

# 4. Convertir NotRepaired a binaria
df_cb['NotRepaired'] = df_cb['NotRepaired'].map({'yes': 1, 'no': 0, 'unknown': -1})

# 5. Separar features y target
X_cb = df_cb.drop('Price', axis=1)
y_cb = df_cb['Price']

# 6. Identificar columnas categóricas originales
cat_features_cb = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand']

# 7. Dividir conjunto de datos
from sklearn.model_selection import train_test_split
X_train_cb, X_valid_cb, y_train_cb, y_valid_cb = train_test_split(X_cb, y_cb, test_size=0.2, random_state=42)


In [8]:
# 🐱 Entrenamiento con CatBoost Regressor


# 1. Crear pools específicos para CatBoost
train_pool = Pool(data=X_train_cb, label=y_train_cb, cat_features=cat_features_cb)
valid_pool = Pool(data=X_valid_cb, label=y_valid_cb, cat_features=cat_features_cb)

# 2. Inicializar modelo con parámetros base
cat_model = CatBoostRegressor(
    iterations=500,
    depth=6,
    learning_rate=0.1,
    random_seed=42,
    verbose=100,
    loss_function='RMSE'
)

# 3. Entrenamiento y medición de tiempo
start_train_cb = time.time()
cat_model.fit(train_pool)
end_train_cb = time.time()

# 4. Predicción y evaluación
start_pred_cb = time.time()
y_pred_cb = cat_model.predict(valid_pool)
end_pred_cb = time.time()

rmse_cb = np.sqrt(mean_squared_error(y_valid_cb, y_pred_cb))

# 5. Mostrar resultados
print("🐱 CatBoost Regressor")
print(f"🔧 Hiperparámetros: iterations=500, depth=6, learning_rate=0.1")
print(f"🧠 RECM (validación): {rmse_cb:.2f}")
print(f"⏱️ Tiempo de entrenamiento: {end_train_cb - start_train_cb:.2f} s")
print(f"⏱️ Tiempo de predicción: {end_pred_cb - start_pred_cb:.4f} s")




0:	learn: 4223.5588475	total: 476ms	remaining: 3m 57s
100:	learn: 1903.0202646	total: 30.9s	remaining: 2m 2s
200:	learn: 1826.2757153	total: 59.3s	remaining: 1m 28s
300:	learn: 1787.3068447	total: 1m 26s	remaining: 57.2s
400:	learn: 1760.8631807	total: 1m 54s	remaining: 28.2s
499:	learn: 1740.0509672	total: 2m 22s	remaining: 0us
🐱 CatBoost Regressor
🔧 Hiperparámetros: iterations=500, depth=6, learning_rate=0.1
🧠 RECM (validación): 1796.27
⏱️ Tiempo de entrenamiento: 143.34 s
⏱️ Tiempo de predicción: 0.1129 s


## Análisis del modelo

In [9]:
# Comparar resultados entre modelos entrenados
comparacion = pd.DataFrame({
    'Modelo': ['Random Forest', 'CatBoost'],
    'RECM': [round(rmse, 2), round(rmse_cb, 2)],
    'Tiempo de entrenamiento (s)': [round(end_train - start_train, 2), round(end_train_cb - start_train_cb, 2)],
    'Tiempo de predicción (s)': [round(end_pred - start_pred, 4), round(end_pred_cb - start_pred_cb, 4)]
})

# Mostrar tabla comparativa
print("\n📊 Comparación de modelos:\n")
print(comparacion)



📊 Comparación de modelos:

          Modelo     RECM  Tiempo de entrenamiento (s)  \
0  Random Forest  2044.23                        25.67   
1       CatBoost  1796.27                       143.34   

   Tiempo de predicción (s)  
0                    0.2765  
1                    0.1129  


Aunque el Random Forest es bastante sólido y rápido, el modelo de CatBoost logró una mejor precisión general (RECM más bajo), aunque con un mayor tiempo de entrenamiento. En resumen, si priorizamos la calidad de predicción, CatBoost es la mejor opción; pero si el tiempo de cómputo es limitado, Random Forest sigue siendo un competidor muy competente.

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [X]  El código no tiene errores
- [X]  Las celdas con el código han sido colocadas en orden de ejecución
- [X]  Los datos han sido descargados y preparados
- [X]  Los modelos han sido entrenados
- [X]  Se realizó el análisis de velocidad y calidad de los modelos